<center>
    
# **Análisis de Bitcoin**

</center>


## **Introducción**

##### este proyecto analiza las caídas del precio de Bitcoin en relación con su valor más alto reciente

##### Supongamos que el precio de Bitcoin sube a 30,000 USD (esto es un pico local), y luego cae un 10%. Cuando el precio cae un 10% desde ese pico, el sistema comenzará a contar los días de caída. Si en algún momento el precio de Bitcoin vuelve a superar los 30,000 USD, el sistema considerará que hubo un nuevo pico local y reiniciará el conteo de días. El objetivo de este proyecto es ayudar a entender cómo de largo es el tiempo entre caídas significativas del Bitcoin y cómo su precio ha evolucionado a lo largo del tiempo en función de esos picos.

## **Setup**

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## **exploración del set de datos**

In [2]:
btc_df = pd.read_csv("BTC.csv")
btc_df

,ticker,date,open,high,low,close
0,BTC,2010-07-17,0.04951,0.04951,0.04951,0.04951
1,BTC,2010-07-18,0.04951,0.08585,0.04951,0.08584
2,BTC,2010-07-19,0.08584,0.09307,0.07723,0.08080
3,BTC,2010-07-20,0.08080,0.08181,0.07426,0.07474
4,BTC,2010-07-21,0.07474,0.07921,0.06634,0.07921
...,...,...,...,...,...,...
5255,BTC,2024-12-05,98298.20000,104062.00000,93081.30000,96751.80000
5256,BTC,2024-12-06,96789.00000,102079.00000,95560.60000,100648.00000
5257,BTC,2024-12-07,100648.00000,100724.00000,99087.60000,99717.70000
5258,BTC,2024-12-08,99717.70000,100548.00000,98767.20000,100276.00000


## **Transformacion**

##### Para nuestro análisis, sólo necesitaremos la columna "close". También vamos a crear la columna "days" la cual va a ser utilizada para nuestra funcion a continucion

In [5]:
btc_df = btc_df.set_index("date")
btc_df = btc_df[["close"]]
btc_df.loc[:, "days"] = 0
btc_df

,close,days
date,,
2010-07-17,0.04951,0
2010-07-18,0.08584,0
2010-07-19,0.08080,0
2010-07-20,0.07474,0
2010-07-21,0.07921,0
...,...,...
2024-12-05,96751.80000,0
2024-12-06,100648.00000,0
2024-12-07,99717.70000,0


### *Funcion para contar la cantidad de dias en caida del precio del Bitcoin*

##### En esta sección vamos a definir una función con el objetivo  de contar los dias transcurridos de caida del precio desde su ultimo pico, vamos a identificar un pico en el precio y lo vamos a comparar con los posteriores tambien.
##### Si se detecta una caída porcentual específica (en relación con el último pico local), el conteo de días se reinicia junto con el pico local, y la evaluación comienza nuevamente desde ese punto.


In [6]:
def drawdown_counter(df, percent):
    peak_price = df["close"].iloc[0] # El peak_price es el precio máximo más reciente . No necesariamente es el precio histórico más alto, sino un "pico local" desde el que se mide la caída.
    
    for i in range(1, len(df)):
        current_price = df["close"].iloc[i]
        previous_count = df["days"].iloc[i-1]
        
        # Si el precio actual (current_price) es mayor al peak_price significa que se está alcanzando un nuevo pico; reemplazo el peak_price por el actual, y sumo 1 al conteo
        if current_price >= peak_price:
            peak_price = current_price
            df.loc[df.index[i], "days"] = previous_count + 1
        
       
        else:
            # Si el precio actual es menor al peak_price, se calcula el porcentaje de caída respecto al último pico local
            percent_diff = abs((peak_price - current_price) / peak_price) * 100
            if percent_diff < percent:
                 #Si la caída porcentual es menor que el porcentaje especificado (percent), se suma 1 al conteo de días.
                df.loc[df.index[i], "days"] = previous_count + 1
            else:
                # Si la caída porcentual es mayor o igual, se reinicia el conteo de días a 0 y el peak_price se actualiza al precio actual.
                df.loc[df.index[i], "days"] = 0
                peak_price = current_price
    
    return df

In [7]:
# Plotly chart, we will start with a 10 percent decline check

percent = 10
df = drawdown_counter(btc_df, percent)

updt = df.index.max()
last = df.iloc[-1]["close"].round(2)
days = int(df.iloc[-1]["days"])

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Creating trace for BTC price data
fig.add_trace(
    go.Scatter(
        x = df.index,
        y = df["close"],
        mode = "lines",
        name = "BTC Price",
        marker = dict(color = 'rgba(0, 0, 0, 1)')),
    secondary_y = False,
)

# Creating trace for a decline counter
fig.add_trace(
    go.Scatter(
        x = df.index,
        y = df["days"],
        mode = "lines",
        name = "Days Since Selloff",
        marker = dict(color = 'rgba(0, 0, 255, 0.4)')),
    secondary_y = True,
)

fig.update_layout(
    template = "plotly_white",
    title = f"BTC and Days Since {percent}% Drawdown<br><sup>Latest Value: {last} ({updt}); {days} days</sup>",
    yaxis_showgrid = False
)

# Set y-axes titles and log scale
fig.update_yaxes(title_text = "Price [USD]", secondary_y = False, type = "log")
fig.update_yaxes(title_text = "Days", secondary_y = True)

fig.show()

### Let's try different percentage values.

### caidas de un 40%

In [8]:
percent = 40
df = drawdown_counter(btc_df, percent)

updt = df.index.max()
last = df.iloc[-1]["close"].round(2)
days = int(df.iloc[-1]["days"])

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Creating trace for BTC price data
fig.add_trace(
    go.Scatter(
        x = df.index,
        y = df["close"],
        mode = "lines",
        name = "BTC Price",
        marker = dict(color = 'rgba(0, 0, 0, 1)')),
    secondary_y = False,
)

# Creating trace for a decline counter
fig.add_trace(
    go.Scatter(
        x = df.index,
        y = df["days"],
        mode = "lines",
        name = "Days Since Selloff",
        marker = dict(color = 'rgba(0, 0, 255, 0.4)')),
    secondary_y = True,
)

fig.update_layout(
    template = "plotly_white",
    title = f"BTC and Days Since {percent}% Drawdown<br><sup>Latest Value: {last} ({updt}); {days} days</sup>",
    yaxis_showgrid = False
)

# Set y-axes titles and log scale
fig.update_yaxes(title_text = "Price [USD]", secondary_y = False, type = "log")
fig.update_yaxes(title_text = "Days", secondary_y = True)

fig.show()

### caidas de un 50%

In [9]:
percent = 50
df = drawdown_counter(btc_df, percent)

updt = df.index.max()
last = df.iloc[-1]["close"].round(2)
days = int(df.iloc[-1]["days"])

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Creating trace for BTC price data
fig.add_trace(
    go.Scatter(
        x = df.index,
        y = df["close"],
        mode = "lines",
        name = "BTC Price",
        marker = dict(color = 'rgba(0, 0, 0, 1)')),
    secondary_y = False,
)

# Creating trace for a decline counter
fig.add_trace(
    go.Scatter(
        x = df.index,
        y = df["days"],
        mode = "lines",
        name = "Days Since Selloff",
        marker = dict(color = 'rgba(0, 0, 255, 0.4)')),
    secondary_y = True,
)

fig.update_layout(
    template = "plotly_white",
    title = f"BTC and Days Since {percent}% Drawdown<br><sup>Latest Value: {last} ({updt}); {days} days</sup>",
    yaxis_showgrid = False
)

# Set y-axes titles and log scale
fig.update_yaxes(title_text = "Price [USD]", secondary_y = False, type = "log")
fig.update_yaxes(title_text = "Days", secondary_y = True)

fig.show()